# Working with many files

In `1-singlefile.ipynb` we learned how to extract subsets and reproject a single image using a variety of tools (GDAL, rasterio, xarray, rioxarray, and holoviz). Often you want to work with a whole stack of imagery - for example let's see how to create a timeseries of backscatter over [Jakobshavn_Glacier](https://en.wikipedia.org/wiki/Jakobshavn_Glacier).

GDAL VRT (Virtual Dataset) files are useful to construct mosaics or sets of multiband imagery https://gdal.org/programs/gdalbuildvrt.html. We first need to take our file list and append GDAL's `/vsicurl/` prefix to all the file names

In [ ]:
import os

In [ ]:
!head -n 2 gamma0.txt

In [ ]:
output = 'vrt-list.txt'

with open('gamma0.txt', 'r') as f:
    gammas = f.readlines()
    vsis = ['/vsicurl/' + line for line in gammas]
    files = [os.path.basename(line) for line in gammas]
    print(f'Saving {len(vsis)} URLs to file list...')
    with open(output, 'w') as v:
        v.writelines(vsis)

!head -n 2 vrt-list.txt

In [ ]:
# Step1, create a GDAL 'VRT' file that lists everything we'd like to work with:
env_vars = 'GDAL_DISABLE_READDIR_ON_OPEN=EMPTY_DIR GDAL_HTTP_COOKIEFILE=.urs_cookies GDAL_HTTP_COOKIEJAR=.urs_cookies'
cog = gammas[1]
cmd = f'{env_vars} gdalbuildvrt -overwrite -allow_projection_difference -separate -input_file_list {output} stack_gamma0.vrt '
print(cmd)

In [ ]:
#%%time 
# this takes ~1min, b/c every file CRS is checked and reprojected if necessary
#!{cmd}

In [ ]:
# it's helpful to add the filename to the description for each band. you can do this with rasterio
import rasterio

# An alternative to using rasterio.Env() is to set global environment variables:
os.environ['GDAL_DISABLE_READDIR_ON_OPEN']='EMPTY_DIR'
os.environ['GDAL_HTTP_COOKIEFILE']='.urs_cookies' 
os.environ['GDAL_HTTP_COOKIEJAR']='.urs_cookies'


with rasterio.open('stack_gamma0.vrt', 'r+') as src:
    print(src.profile)
    src.descriptions = files

In [ ]:
# Open the VRT with xarray (this is very fast b/c only metadata is read!)
import xarray as xr
da = xr.open_rasterio('stack_gamma0.vrt', chunks=dict(band=1, x=532200, y=512)) #ensure data loaded as dask arrays
da.data

In [ ]:
print(f'Total uncompressed dataset size= {da.nbytes/1e12} TB')

In [ ]:
# Assign a time coordinate instead of integer band
#import pandas as pd
#dates = [url.split('/')[-2] for url in gammas]
#print(dates[:2])

#datetimes = [pd.to_datetime(x) for x in dates]
#datetimes[:2]

In [ ]:
# make 'time' active coordinate instead of integer band
#da = da.assign_coords(time=('band', datetimes))
#da = da.swap_dims({'band':'time'})
#da.name = 'gamma0'
#da

In [ ]:
# Bounding box of interest 
# draw on here: http://geojson.io/
import geopandas as gpd
import hvplot.pandas
gf = gpd.read_file('jakobshavn.geojson')
bbox = gf.hvplot.polygons(alpha=0.2, geo=True, tiles=True)
bbox

In [ ]:
# convert our bounding box to epsg:3413 (south polar sterographic)
# NOTE: miny and maxy are switched here...
gf3413 = gf.to_crs(3413)
gf3413.bounds

In [ ]:
xmin, ymax, xmax, ymin = gf3413.bounds.values[0]

In [ ]:
# Now extract that subset from all the GIMP data
import hvplot.xarray
subset = da.sel(x=slice(xmin, xmax), y=slice(ymin, ymax))
subset.data

In [ ]:
print(f'Subset uncompressed dataset size= {subset.nbytes/1e9} GB')

In [ ]:
%%time 
# We can hold 6GB of data in memory, so let's do that to make plotting faster

# subset.persist() # RasterioIOError: Read or write failed. HTTP response code: 503
#ds = subset.compute() # RasterioIOError: Read or write failed. Request for 1524742125-1525203366 failed


# seems like lots of trouble reading these geotiffs

In [ ]:
# Let's just try saving data to a local netcdf
# more network errors...
#subset.to_netcdf('mysubset.nc')

In [ ]:
#subset.hvplot.image(rasterize=True, dynamic=True, frame_width=400, aspect='equal', cmap='gray')
# occaisional network error: RasterioIOError: Read or write failed. /vsicurl/https://n5eil01u.ecs.nsidc.org/DP4/MEASURES/NSIDC-0723.003/2015.01.01/GL_S1bks_mosaic_01Jan15_12Jan15_gamma0_50m_v03.0.tif, band 1: IReadBlock failed at X offset 14, Y offset 61: TIFFReadEncodedTile() failed.

In [ ]:
#%%time 

# if 'time' is default coordinate
# WARNING:param.Image03243: Image dimension time is  not evenly sampled to relative tolerance of 0.001. Please use the QuadMesh element for irregularly sampled data or set a higher tolerance on hv.config.image_rtol or the rtol parameter in the Image constructor.
# UFuncTypeError: ufunc 'true_divide' cannot use operands with types dtype('float64') and dtype('<m8[ns]')
# something weird with time


# Plot just the subset (? does this still read the entire file rather than just an overview)
#subset.hvplot.image(rasterize=True, dynamic=True, frame_width=400, aspect='equal', cmap='gray')